<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# 设置消息阈值
tf.logging.set_verbosity(tf.logging.INFO)

In [ ]:
# 设置参数和读取数据

In [ ]:
model_params = {"learning_rate": 0.01}

mnist = input_data.read_data_sets("../../../TensorFlow/datasets/MNIST_data/", one_hot=False)

In [3]:
# 构建模型

# 计算前向传播
def lenet(x, is_training):
    x = tf.placeholder(x, shape=[-1, 28, 28, 1])
    
    conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
    conv1 = tf.layers.max_pooling2d(conv1, 2, 2)
    
    conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
    conv2 = tf.layers.max_pooling2d(conv2, 2, 2)
    
    fc1 = tf.contrib.layers.flatten(conv2)
    fc1 = tf.layers.dense(fc1, 1024) # 添加全连接层
    fc1 = tf.layers.dropout(fc1, rate=0.4, training=is_training)
    result = tf.layers.dense(fc1, 10) # 添加全连接层
    
    return result

def model_fn(features, label, mode, params):
    # 计算前向传播过程
    predict = lenet(features["image"], mode==tf.estimator.ModeKeys.TRAIN)
    
    # 如果是作为预测, 则返回预测结果
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions={"result": tf.argmax(predict, 1)})
    
    # 计算损失函数
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=predict, labels=labels))
    
    # 定义优化方法
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=params["learning_rate"])
    
    # 计算精度
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(tf.argmax(predict, 1)), labels}
    
    # 返回计算结果
    return tf.estimator.EstimatorSpec(mode=mode, 
                                      loss=loss, 
                                      train_op=train_op, 
                                      eval_metric_ops=eval_metric_ops)

# 定义模型
estimator = tf.estimator.Estimator(model_fn=model_fn, params=model_params)

In [ ]:
# 进行训练

In [ ]:
# 定义训练输入数据
train_input_op = tf.estimator.inputs.numpy_input_fn(x={"image":mnist.train.images}, 
                                                    y=mnist.train.labels.astype(np.int32), 
                                                    num_epochs=None, 
                                                    batch_size=128, 
                                                    shuffle=True)

# 进行训练
estimator.train(input_fn=train_input_fn, steps=30000)

In [ ]:
# 测试模型

In [ ]:
# 定义测试数据
test_input_fn = tf.estimator.inputs.numpy_input_fn(x={"image":mnist.test.images}, 
                                                   y=mnist.test.labels.astype(np.int32), 
                                                   num_epochs=1, 
                                                   batch_size=128, 
                                                   shuffle=False)

# 进行测试
test_results = estimator.evaluate(input_fn=test_input_fn)
accuracy_score = test_results["accuracy"]
print("\nTest accuracy: %g %%" % (accuracy_score*100))

In [ ]:
# 进行预测

In [ ]:
# 定义预测数据
predict_input_fn = tf.estimator.inputs.numpy_input_fn(x={"image":mnist.test.images["10"]}, 
                                                      num_epochs=1, 
                                                      shuffle=False)

# 进行预测
predictions = estimator.predict(input_fn=predict_input_fn)
for i, p in enumerate(predictions):
    print("Prediction %s: %s" % (i+1, p['result']))